In [50]:
#Import dependencies
import quandl
import numpy as np 
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR
from sklearn.model_selection import train_test_split

In [51]:
#Retrieve data from Quandl
quandl.ApiConfig.api_key = "1sNXfWjF5c7B4CLEZZyT"

In [52]:
#Assign start and end dates form the stock
start = pd.to_datetime('2014-11-17')
end = pd.to_datetime('2019-11-17')
data_TGT = quandl.get('EOD/TGT', start_date='2014-11-17', end_date='2019-11-17')
data_TGT.head()

,Open,High,Low,Close,Volume,Dividend,Split,Adj_Open,Adj_High,Adj_Low,Adj_Close,Adj_Volume
Date,,,,,,,,,,,,
2014-11-17,67.58,67.70,66.90,67.13,7260198.0,0.52,1.0,57.568198,57.670420,56.988938,57.184865,7260198.0
2014-11-18,66.96,67.76,66.50,67.51,5581483.0,0.00,1.0,57.040050,57.721532,56.648197,57.508569,5581483.0
2014-11-19,69.95,73.08,69.86,72.50,25614518.0,0.00,1.0,59.587089,62.253387,59.510422,61.759313,25614518.0
2014-11-20,71.34,71.83,70.30,71.19,11064661.0,0.00,1.0,60.771164,61.188572,59.885237,60.643386,11064661.0
2014-11-21,71.80,72.26,71.05,71.51,6835245.0,0.00,1.0,61.163016,61.554868,60.524127,60.915979,6835245.0


In [58]:
# Get the Adjusted Close Price 
data_TGT = data_TGT[['Adj_Close']] 
# Take a look at the new data 
print(data_TGT.head())

            Adj_Close
Date                 
2014-11-17  57.184865
2014-11-18  57.508569
2014-11-19  61.759313
2014-11-20  60.643386
2014-11-21  60.915979


In [59]:
# A variable for predicting 'n' days out into the future
forecast_out=30     
#Create another column (the target ) shifted 'n' units up
data_TGT['Prediction'] = data_TGT[['Adj_Close']].shift(-forecast_out)
#print the new data set
print(data_TGT.tail())


            Adj_Close  Prediction
Date                             
2019-11-11     110.45         NaN
2019-11-12     108.35         NaN
2019-11-13     108.58         NaN
2019-11-14     111.11         NaN
2019-11-15     113.21         NaN


In [60]:
### Create the independent data set (X)  #######
# Convert the dataframe to a numpy array
X = np.array(data_TGT.drop(['Prediction'],1))

#Remove the last '30' rows
X = X[:-forecast_out]
print(X)

[[ 57.18486452]
 [ 57.5085685 ]
 [ 61.75931294]
 ...
 [105.16      ]
 [106.62      ]
 [109.1       ]]


In [61]:
### Create the dependent data set (y)  #####
# Convert the dataframe to a numpy array 
y = np.array(data_TGT['Prediction'])
# Get all of the y values except the last '30' rows
y = y[:-forecast_out]
print(y)

[ 64.66413028  64.17005578  63.02005478 ... 108.58       111.11
 113.21      ]


In [62]:
# Split the data into 80% training and 20% testing
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [63]:
# Create and train the Support Vector Machine (Regressor) 
svr_rbf = SVR(kernel='rbf', C=1e3, gamma=0.1) 
svr_rbf.fit(x_train, y_train)

SVR(C=1000.0, cache_size=200, coef0=0.0, degree=3, epsilon=0.1, gamma=0.1,
  kernel='rbf', max_iter=-1, shrinking=True, tol=0.001, verbose=False)

In [64]:
# Testing Model: Score returns the coefficient of determination R^2 of the prediction. 
# The best possible score is 1.0
svm_confidence = svr_rbf.score(x_test, y_test)
print("svm confidence: ", svm_confidence)

svm confidence:  0.6761840993544943


In [65]:
# Create and train the Linear Regression  Model
lr = LinearRegression()
# Train the model
lr.fit(x_train, y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None,
         normalize=False)

In [66]:
# Testing Model: Score returns the coefficient of determination R^2 of the prediction. 
# The best possible score is 1.0
lr_confidence = lr.score(x_test, y_test)
print("lr confidence: ", lr_confidence)

lr confidence:  0.705700437498372


In [67]:
# Set x_forecast equal to the last 30 rows of the original data set from Adj. Close column
x_forecast = np.array(data_TGT.drop(['Prediction'],1))[-forecast_out:]
print(x_forecast)

[[107.95]
 [108.52]
 [110.36]
 [110.57]
 [111.83]
 [111.15]
 [111.89]
 [112.19]
 [113.23]
 [112.81]
 [113.68]
 [113.19]
 [111.36]
 [110.25]
 [109.02]
 [109.72]
 [108.15]
 [107.93]
 [106.91]
 [107.82]
 [108.77]
 [110.06]
 [110.51]
 [110.59]
 [110.15]
 [110.45]
 [108.35]
 [108.58]
 [111.11]
 [113.21]]


In [68]:
# Print linear regression model predictions for the next '30' days
lr_prediction = lr.predict(x_forecast)
print(lr_prediction)

[107.71783174 108.26972667 110.05128224 110.25461195 111.47459022
 110.81618925 111.53268442 111.82315544 112.83012163 112.42346221
 113.26582815 112.79139216 111.01951896 109.9447762  108.75384503
 109.43161074 107.91147909 107.69846701 106.71086555 107.59196097
 108.51178585 109.76081122 110.19651775 110.27397668 109.84795253
 110.13842354 108.10512643 108.32782088 110.77745978 112.81075689]


In [69]:
# Print support vector regressor model predictions for the next '30' days
svm_prediction = svr_rbf.predict(x_forecast)
print(svm_prediction)

[109.13309762 109.50970341 112.37556475 111.47251294  98.05973599
 106.76539851  97.20369903  92.88826138  80.07857457  84.59057258
  76.53696287  80.45948441 104.35225188 112.67941955 110.78623391
 112.73762346 109.12286857 109.14169452 110.33164373 109.2106319
 110.06242419 112.94823333 111.77466597 111.3638783  112.8597552
 112.04133451 109.26446516 109.62273733 107.18729021  80.26769228]
